CoinAPI (https://www.coinapi.io/) was the source used for cryptocurrency historical data crawling. A free API key supports max 100 daily queries. 3 main interfaces (REST, WebSocket and FIX) can be used to access data while I chose REST in this case. 

In [2]:
import click
import os
import pandas as pd
import random
import time
import urllib.request as urlreq
import requests
from datetime import datetime

DATA_DIR="data"
RANDOM_SLEEP_TIMES=(1,5)
CRYPTOCURRENCY_LIST_URL="https://api.coinmarketcap.com/v1/ticker/?limit=50"
CRYPTOCURRENCY_LIST_PATH = os.path.join(DATA_DIR, "cryptocurrency_list.csv")

Top 50 crypocurrency symbols get fetched and saved in "data/cryptocurrency_list.csv".

In [3]:
def save_symbol():
    if os.path.exists(CRYPTOCURRENCY_LIST_PATH):
        return
    response=requests.get(CRYPTOCURRENCY_LIST_URL)
    print(response.status_code)
    data=response.json()
    
    with open(CRYPTOCURRENCY_LIST_PATH,'w') as fin:
        print('symbol', file=fin)
        for d in data:
            print(d['symbol'],file=fin)
    return

In [4]:
save_symbol()

In [5]:
def load_symbols():
    save_symbol()
    df_symbols=pd.read_csv(CRYPTOCURRENCY_LIST_PATH)
    #print(df_symbols)
    cryptocurrency_symbols=df_symbols['symbol'].unique().tolist()
    #print(cryptocurrency_symbols)
    print ("Loaded %d stock symbols", len(cryptocurrency_symbols))
    return cryptocurrency_symbols

In [6]:
load_symbols()

Loaded %d stock symbols 50


['BTC',
 'ETH',
 'XRP',
 'BCH',
 'LTC',
 'EOS',
 'ADA',
 'NEO',
 'XLM',
 'MIOTA',
 'XMR',
 'DASH',
 'XEM',
 'TRX',
 'USDT',
 'ETC',
 'VEN',
 'QTUM',
 'ICX',
 'LSK',
 'OMG',
 'NANO',
 'BTG',
 'BNB',
 'ZEC',
 'DGD',
 'PPT',
 'STEEM',
 'STRAT',
 'WAVES',
 'BCN',
 'XVG',
 'MKR',
 'SC',
 'BTS',
 'RHOC',
 'DOGE',
 'AE',
 'SNT',
 'REP',
 'DCR',
 'AION',
 'BTM',
 'WTC',
 'ONT',
 'KMD',
 'ZIL',
 'ARDR',
 'ZRX',
 'HSR']

In [8]:
import urllib.parse
import csv
import json
def fetch_prices(symbol,out_name):
    now_datetime=datetime.now().strftime("%Y-%m-%d")
    BASE_URL = "https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_{0}_USD/history?apikey=B0535108-9739-4816-81F4-4F72BF51015A&period_id=1HRS&limit=100000&time_start=2016-01-01T00:00:00&time_end={1}T00:00:00"
    symbol_url=BASE_URL.format(
        urllib.parse.quote(symbol),
        urllib.parse.quote(now_datetime)
    )
    print ("Fetching {} ...".format(symbol))
    print (symbol_url)
    try:
        response = requests.get(symbol_url)
        print(response.status_code)
        data=response.json()
        headers=list(data[0].keys())
        with open(out_name,'w+') as csvfile:
            writer = csv.writer(csvfile, delimiter=',', quotechar='"',lineterminator='\n', quoting=csv.QUOTE_MINIMAL)
            writer.writerow(h for h in headers)
            for d in data:
                writer.writerow(d.values())
    except: 
        print(response.status_code)
        return False
    
    readout=pd.read_csv(out_name)
    if readout.empty:
        print("Remove {} because the data set is empty.".format(out_name))
        os.remove(out_name)
    else:
        dates=readout.iloc[:,0].tolist()
        print ("# Fetched rows: %d [%s to %s]" % (readout.shape[0], dates[-1], dates[0]))
    
    sleep_time=random.randint(*RANDOM_SLEEP_TIMES)
    print("Sleeping... %ds" %sleep_time)
    time.sleep(sleep_time)
    return True

In [11]:
def main():
    random.seed(time.time())
    num_failure=0
    
    symbols=load_symbols()
    for idx, sym in enumerate(symbols):
        out_name=os.path.join(DATA_DIR,sym+"1HRS.csv")
        if os.path.exists(out_name):
            print("Fetched",sym)
            continue
        succeeded=fetch_prices(sym,out_name)
        num_failure+= int(not succeeded)
        if idx%10==0:
            print("#Failures so fat [%d/%d]: %d" %(idx+1,len(symbols),num_failure))


In [12]:
main()

Loaded %d stock symbols 50
Fetched BTC
Fetched ETH
Fetched XRP
Fetching BCH ...
https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_BCH_USD/history?apikey=B0535108-9739-4816-81F4-4F72BF51015A&period_id=1HRS&limit=100000&time_start=2016-01-01T00:00:00&time_end=2018-04-12T00:00:00
200
# Fetched rows: 2447 [2018-04-11T23:00:00.0000000Z to 2017-12-28T21:00:00.0000000Z]
Sleeping... 2s
Fetching LTC ...
https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_LTC_USD/history?apikey=B0535108-9739-4816-81F4-4F72BF51015A&period_id=1HRS&limit=100000&time_start=2016-01-01T00:00:00&time_end=2018-04-12T00:00:00
200
# Fetched rows: 6891 [2018-04-11T23:00:00.0000000Z to 2017-06-28T14:00:00.0000000Z]
Sleeping... 1s
Fetching EOS ...
https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_EOS_USD/history?apikey=B0535108-9739-4816-81F4-4F72BF51015A&period_id=1HRS&limit=100000&time_start=2016-01-01T00:00:00&time_end=2018-04-12T00:00:00
400
400
Fetching ADA ...
https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_ADA_USD/history?apikey=

400
400
Fetching DCR ...
https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_DCR_USD/history?apikey=B0535108-9739-4816-81F4-4F72BF51015A&period_id=1HRS&limit=100000&time_start=2016-01-01T00:00:00&time_end=2018-04-12T00:00:00
400
400
#Failures so fat [41/50]: 36
Fetching AION ...
https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_AION_USD/history?apikey=B0535108-9739-4816-81F4-4F72BF51015A&period_id=1HRS&limit=100000&time_start=2016-01-01T00:00:00&time_end=2018-04-12T00:00:00
400
400
Fetching BTM ...
https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_BTM_USD/history?apikey=B0535108-9739-4816-81F4-4F72BF51015A&period_id=1HRS&limit=100000&time_start=2016-01-01T00:00:00&time_end=2018-04-12T00:00:00
400
400
Fetching WTC ...
https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_WTC_USD/history?apikey=B0535108-9739-4816-81F4-4F72BF51015A&period_id=1HRS&limit=100000&time_start=2016-01-01T00:00:00&time_end=2018-04-12T00:00:00
400
400
Fetching ONT ...
https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_ONT_USD/history?a

You may find lots of cryptocurrencies historical data not fetched with a status_code: 400. Because we used BITSTAMP as the exchange platform, which only exchange a small list of cryptocurrencies. You can accordingly adjust the API queries to fetch the symbol you like from a matching exchange platform.